In [1]:
from DDPG import ddpg
from environment.vehicleEnv import VehicleEnv

2022-07-14 23:03:14.724058: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-14 23:03:14.921096: E tensorflow/stream_executor/cuda/cuda_blas.cc:2982] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-07-14 23:03:15.747829: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-07-14 23:03:15.747987: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

# Parameters

In [2]:

# vehicles
LEAD_VELOCITY = 27.78
LEAD_POSITION = 100.0
INITIAL_EGO_VELOCITY = 15.0
EGO_POSITION = 0.0
SET_EGO_VELOCITY = 30

SAFETY_DISTANCE = 50
# INITIAL_GAP = 100

# network
GAMMA = 0.99
BATCH_SIZE = 64
MEMORY_SIZE = 500000
LEARNING_RATE_ACTOR = 0.001
LEARNING_RATE_CRITIC = 0.0001
TAU = 0.001
NOISE_MEAN = 0
NOISE_SIGMA = 0.02

INPUT_SHAPE = (2,)

# simulation
N_SIMULATIONS = 5000

# Model

In [3]:
# initialize networks
agent = ddpg.Agent(
            input_shape=INPUT_SHAPE, 
            alpha=LEARNING_RATE_ACTOR, 
            beta=LEARNING_RATE_CRITIC, 
            gamma=GAMMA, 
            n_actions=1, 
            max_size=MEMORY_SIZE, 
            tau=TAU, 
            batch_size=BATCH_SIZE, 
            noise=NOISE_SIGMA
            )

2022-07-14 23:03:16.538371: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2022-07-14 23:03:16.538401: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: bobby-OMEN-by-HP-Laptop-15-dh1xxx
2022-07-14 23:03:16.538408: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: bobby-OMEN-by-HP-Laptop-15-dh1xxx
2022-07-14 23:03:16.538468: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 515.48.7
2022-07-14 23:03:16.538489: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 515.48.7
2022-07-14 23:03:16.538494: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 515.48.7
2022-07-14 23:03:16.538759: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the fol

# Simulation

In [4]:
def one_simulation():
    state_list = []
    speed_ego_list = []
    speed_lead_list = []
    action_list = []
    reward_list = []
    

    env = VehicleEnv(
            lead_velocity=LEAD_VELOCITY, 
            lead_position=LEAD_POSITION, 
            ego_velocity=INITIAL_EGO_VELOCITY, 
            ego_position=EGO_POSITION)
    state = env.get_state()
    done = False
    return_episode = 0
    steps = 0
    for i in range(200):
        #print("State: ", state)
        state_list.append(state)
        #print("Speed ego: ", env.ego.get_velocity())
        speed_ego_list.append(env.ego.get_velocity()[0])
        #print("Speed lead: ", env.lead.get_velocity())
        speed_lead_list.append(env.lead.get_velocity()[0])
        # Select an action
        action = agent.choose_action(state)
         
        # Perform the action
        next_state, reward, done = env.step(action)
        
        assert type(env.ego.get_velocity()) != int

       
        #print("Action: ", action)
        action_list.append(action)
        #print("Next state: ", next_state)
        #print("Reward: ", reward)
        reward_list.append(reward)
        #print("Done: ", done)
        
        # Store the transition
        agent.remember(state, action, reward, next_state, done)
                
        # Training
        agent.learn()
                
        # Go in the next state
        state = next_state
                
        # Increment time
        # steps += 1
        return_episode += reward
        steps += 1
        if done:
            break
    return return_episode, steps, reward_list# , state_list, speed_ego_list, speed_lead_list, action_list,

In [5]:
import utils
return_episode = []
steps_episode = []
reward_episode = []

In [6]:
import time
num = 0
i = 0
while num < 1000000:
    tic = time.perf_counter()
    return_episod, steps, reward_list = one_simulation()
    return_episode.append(return_episod)
    steps_episode.append(steps)
    reward_episode.append(reward_list)
    print(i)
    i+=1
    num+=steps
    print("steps: ", num)
    tac = time.perf_counter()
    print(f"{tac-tic:0.4f} seconds")

0
steps:  200
14.9529 seconds
1
steps:  400
20.1009 seconds
2
steps:  600
20.4736 seconds
3
steps:  800
20.7154 seconds


KeyboardInterrupt: 

In [ ]:
import utils

In [ ]:
def test_simulation():
    state_list = []
    speed_ego_list = []
    speed_lead_list = []
    action_list = []
    reward_list = []
    

    env = VehicleEnv(
            lead_velocity=LEAD_VELOCITY, 
            lead_position=LEAD_POSITION, 
            ego_velocity=INITIAL_EGO_VELOCITY, 
            ego_position=EGO_POSITION)
    state = env.get_state()
    done = False
    return_episode = 0
    steps = 0
    for i in range(200):
        #print("State: ", state)
        state_list.append(state)
        #print("Speed ego: ", env.ego.get_velocity())
        speed_ego_list.append(env.ego.get_velocity()[0])
        #print("Speed lead: ", env.lead.get_velocity())
        speed_lead_list.append(env.lead.get_velocity()[0])
        # Select an action
        action = agent.choose_action(state)
         
        # Perform the action
        next_state, reward, done = env.step(action)
        
        assert type(env.ego.get_velocity()) != int

       
        #print("Action: ", action)
        action_list.append(action)
        #print("Next state: ", next_state)
        #print("Reward: ", reward)
        reward_list.append(reward)
        #print("Done: ", done)
        
        # Store the transition
        agent.remember(state, action, reward, next_state, done)
                
        # Training
        agent.learn()
                
        # Go in the next state
        state = next_state
                
        # Increment time
        # steps += 1
        return_episode += reward
        steps += 1
        if done:
            break
    return state_list, speed_ego_list, speed_lead_list, action_list

state_list, speed_ego_list, speed_lead_list, action_list = test_simulation()

In [ ]:
utils.plot_reward(reward_list, path="graphs")

In [ ]:
utils.plot_control_input(action_list, "graphs")

In [ ]:
utils.plot_speed(speed_ego_list, "graphs")

In [ ]:
utils.plot_undiscounted_episode_reward(return_episode, "graphs")

In [ ]:
import pickle

with open('rewrds.txt', 'wb') as fh:
   pickle.dump(reward_episode, fh)

In [ ]:
import pickle
pickle_off = open ("steps.txt", "rb")
emp = pickle.load(pickle_off)
print(emp)

In [ ]:
agent.save_models()